<a href="https://colab.research.google.com/github/lavanaythakral/GPT2-answers/blob/master/overfit_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data

In [1]:
import pandas as pd
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import pandas as pd
import re
import statistics
import numpy

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [19]:
import scipy
import csv

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/My Drive/Thesis project/Thesis project/datasets/WOF_split_into_sentences.csv')
sentences = list(df['Sentences'])
len(sentences)

3085

#Sentence Transformers

In [4]:
!pip install -U sentence-transformers

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 1.1MB 13.3MB/s 
     |████████████████████████████████| 1.1MB 31.2MB/s 
     |████████████████████████████████| 3.0MB 43.8MB/s 
     |████████████████████████████████| 890kB 55.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.7-cp36-none-any.whl size=91168 sha256=21a48655b7c01048f9c3293a428432ce3e87d068b18a6f687f80ae516440498e
  Stored in directory: /root/.cache/pip/wheels/0f/8d/11/6d3bae62f9735ad3a34eca2df5334d3b6f6e1721692b8ebf21
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=e62ac003dd063ceedbd288ed5bed2bbb57ef60af85c23ab41ddff1ee1ab5164e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:24<00:00, 16.6MB/s]


In [6]:
corpus_embeddings = model.encode(sentences)

In [7]:
len(corpus_embeddings)

3085

In [8]:
#s1,s2 are strings
def calculate(s1,s2):
  e1 = model.encode([s1])
  e2 = model.encode([s2])
  distance = scipy.spatial.distance.cdist([e1[0]], [e2[0]], "cosine")[0]
  return distance[0]

In [9]:
def verification(questions,answers):
  scores = []
  for ques,ans in zip(questions,answers):
    scores.append(calculate(ques,ans))
  return scores

##GPT2

In [10]:
!pip install transformers

In [11]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
gpt2.download_gpt2(model_name="355M")
gpt2.copy_checkpoint_from_gdrive(run_name='run1_topical_token')
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1_topical_token')

Fetching checkpoint: 1.05Mit [00:00, 265Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 75.9Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 547Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:09, 156Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 560Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 74.7Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 131Mit/s]                                                       


Loading checkpoint checkpoint/run1_topical_token/model-2000
INFO:tensorflow:Restoring parameters from checkpoint/run1_topical_token/model-2000


INFO:tensorflow:Restoring parameters from checkpoint/run1_topical_token/model-2000


In [13]:
def generate(input):
  print("GPT2 generating for :",input)
  generated_text = gpt2.generate(sess,
              length=100,
              run_name='run1_topical_token',
              return_as_list=True,
              temperature=0.7,
              prefix=input,
              nsamples=15,
              truncate = '.',
              batch_size=5,
              top_k = 5,
              include_prefix = False)
  
  def clean(input_st, sub):
    return input_st.replace(sub, '').lstrip()

  cleaned = []
  for text in generated_text:
    cleaned.append(re.sub(r"^\W+", "",clean(text,'<|endoftext|>')))

  return cleaned

In [28]:
def check(questions,sentences,corpus_embeddings):
  rows = []
  cols = ['question','average','stdev','min','min response','original sentence','max','max response','original sentence']
  filename = '/content/drive/My Drive/Thesis project/Thesis project/datasets/overfitting_check/stats_bert.csv'
  
  with open(filename,"w") as f:
    writer = csv.writer(f)
    writer.writerow(cols)

  for ques in questions:
    sample = generate(ques)
    sample_embeddings = model.encode(sample)
    closest_n = 1
    scores = []
    for res, sample_embedding in zip(sample, sample_embeddings):
      distances = scipy.spatial.distance.cdist([sample_embedding], corpus_embeddings, "cosine")[0]
      results = zip(range(len(distances)), distances)
      results = sorted(results, key=lambda x: x[1])

      print("\n\n======================\n\n")
      print("Query:", res)
      print("\nmost similar sentence in corpus:")

      for idx, distance in results[0:closest_n]:
        print(sentences[idx].strip(), "(Score: %.4f)" % (1-distance))
        scores.append([1-distance,res,sentences[idx]])
    row = stats(ques,scores)
    rows.append(row)
    
    with open(filename,"a") as f:
      writer = csv.writer(f)
      writer.writerow(row)
  
  return rows
        

In [26]:
def stats(question,answers):
  scores = [x[0] for x in answers]
  average = statistics.mean(scores)
  stdev = statistics.stdev(scores)
  minm = min(scores)
  maxm = max(scores)
  min_response = ""
  sim_sen_min = ""
  max_response = ""
  sim_sen_max = ""

  for sen in answers:
    if(sen[0] == minm):
      min_response = sen[1]
      sim_sen_min = sen[2]
    if(sen[0] == maxm):
      max_response = sen[1]
      sim_sen_max = sen[2]

  return [question,average,stdev,minm,min_response,sim_sen_min,maxm,max_response,sim_sen_max]

In [30]:
test_questions = [
"How can I overcome difficulty?",
"Why did I fail?",
"How can I become successful?",
"How can I be more creative?",
"What is love?",
"Do you need to fail to succeed?",
"Is there an afterlife?",
"What is the meaning of life?",
"How do you judge a person?",
"What’s the most important thing in you life?",
"What’s your definition of success?",
"What type of person do I want to be?",
"How can you love yourself more today?",
"How can you change someone’s life for the better today?",
"What inspire you the most?",
"What should I do next?",
"What should I have for dinner?",
"cheer up",
"what is your mission?"]

In [32]:
check(test_questions,sentences,corpus_embeddings)

GPT2 generating for : How can I overcome difficulty?




Query: I have dealt with it head on,  but it is necessary for all of us to tackle these challenges head on

most similar sentence in corpus:
The whole system — its people, procedures,  infrastructure — would have to learn to extend itself. (Score: 0.7929)




Query: I have dealt with it head on,  but it is essential that we approach it from the correct angle

most similar sentence in corpus:
They would have to be able to sort out things amicably,  and take responsibility for slip-ups. (Score: 0.8307)




Query: There are a few things I do differently from  theamiable Dr Brahm Prakash

most similar sentence in corpus:
I experienced mixed feelings. (Score: 0.7816)




Query: The answer is to  become the sort of man who can count on one hand the number of  people who have overcome difficulties and, in another, the number of  people who have nonetheless  difficulties

most similar sentence in corpus:
I wanted men who had the capabili

[['How can I overcome difficulty?',
  0.76565070915478,
  0.08425301405158957,
  0.6578359814289867,
  'I have only to look at the faces of those who faced it',
  'For  they have their own thoughts".',
  0.9931476386706635,
  'I tried to be  receptive to every constructive idea emanating from my subordinates  and to relay it in an appropriate form for critical examination and  implementation',
  'At the same time, I tried to be  receptive to every constructive idea emanating from my subordinates  and to relay it in an appropriate form for critical examination and  implementation.'],
 ['Why did I fail?',
  0.7835653115399522,
  0.04343991201638026,
  0.6805808818443227,
  'Why did I fail to see the forest for the trees?I had read somewhere,  "Know where you stand, and strive for  whatever it takes to win"',
  'I tried  hard to do as he said, which was to strive to control my thoughts and my  mind and, through these, to influence my destiny Ironically, that destiny  did not lead me back 

In [33]:
df = pd.read_csv('/content/drive/My Drive/Thesis project/Thesis project/datasets/overfitting_check/stats_bert.csv')
df.head(5)

,question,average,stdev,min,min response,original sentence,max,max response,original sentence.1
0,How can I overcome difficulty?,0.765651,0.084253,0.657836,I have only to look at the faces of those who ...,"For they have their own thoughts"".",0.993148,I tried to be receptive to every constructive...,"At the same time, I tried to be receptive to ..."
1,Why did I fail?,0.783565,0.043440,0.680581,Why did I fail to see the forest for the trees...,"I tried hard to do as he said, which was to s...",0.856940,Was it my failure to take up the challenge of ...,"He asked me instead, if we were unenthused by ..."
2,How can I become successful?,0.824033,0.085148,0.704019,The answer lies in the artist and his ability...,Flow is a by-product of controlled creativity.,0.998238,I would like to share with you two techniques...,I would like to share with you two techniques...
3,How can I be more creative?,0.810553,0.063202,0.733098,I was asked this question by Dr Brahm Prakash...,"Dr OP Mediratta, Director of the ADE, asked m...",0.989848,The key is to understand the difference betwe...,One must understand the difference between a ...
4,What is love?,0.785957,0.068664,0.656975,I call it the waking of the spirit,Rev.,0.981638,I ask my father,I asked my father.


In [35]:
average_scores = list(df['average'])
len(average_scores)

19

In [36]:
av = statistics.mean(average_scores)
av

0.7959193408586669